In [11]:
import numpy as np

In [4]:

# type
A=1 # 民
B=2 # 神
L=3 # 狼
Y=4 # 预言家
S=5 # 守卫
H=6 # 猎人（女巫视作猎人+狼人第一回合不能杀人）

# status
U = 0 # 没金水（Unknown)
J = 1 # 金水

In [5]:
# 狼人杀人
def kill(people, mask, turn):
    # 随机找一个人杀
    idxs = np.where(people!=L)[0]
    idx = np.random.choice(idxs)
    
    # 如果杀了守卫守的人，就摸了
    if (turn%2==1 and people[idx]!=S) or (turn%2==0 and people[idx]!=Y):
        
        # 如果杀到了猎人/女巫，就加一个复仇的flag
        flag_revenge = people[idx] == H
        
        # 随机杀一个人
        people = np.concatenate([people[:idx], people[idx+1:]])
        mask = np.concatenate([mask[:idx], mask[idx+1:]])
        
        # 复仇：随机杀一个没金水的
        if flag_revenge:
            idxs = np.where(mask!=J)[0]
            idx = np.random.choice(idxs)
            people = np.concatenate([people[:idx], people[idx+1:]])
            mask = np.concatenate([mask[:idx], mask[idx+1:]])    
    return people, mask

In [6]:
# 票人
# 没金水的抽奖杀一个
def vote(people, mask):
    idxs = np.where(mask!=J)[0]
    idx = np.random.choice(idxs)
    people = np.concatenate([people[:idx], people[idx+1:]])
    mask = np.concatenate([mask[:idx], mask[idx+1:]])
    return people, mask

In [7]:
# 查人
def check(people, mask):
    # 抽个没金水不是自己的
    idxs = np.where(np.logical_and(mask!=J, people!=Y))[0]
    idx = np.random.choice(idxs)
    
    # 查了狼直接秒了，这一天没有投票
    if people[idx] == L:
        people = np.concatenate([people[:idx], people[idx+1:]])
        mask = np.concatenate([mask[:idx], mask[idx+1:]])
        flag = True
    # 反之，发一个金水并投票
    else:
        mask[idx] = J
        flag = False
    return people, flag, mask

In [8]:
# 检查好人是不是死光了（屠边）
def min_all_killed(people):
    return A not in people or (H not in people and Y not in people and S not in people)

In [9]:
# monte carlo
import tqdm

In [13]:
cnt_L = 0
cnt_M = 0
for i in tqdm.tqdm(range(100000)):
    # 四民四神四狼
    people = np.array([A,A,A,A,H,H,S,Y,L,L,L,L])
    # 开始啥也不知道
    mask   = np.array([U,U,U,U,U,U,U,U,U,U,U,U])
    
    for turn in range(1, 100):
        # 检查结束，下同
        if min_all_killed(people):
            cnt_L += 1
            break
        if L not in people:
            cnt_M += 1
            break
        # 第一个回合不死人
        if turn > 1:
            people, mask = kill(people, mask, turn)
        
        if min_all_killed(people):
            cnt_L += 1
            break
        if L not in people:
            cnt_M += 1
            break
        
        successfully_checked = False
        # 如果预言家活着，预言家查人
        if Y in people:
            people, successfully_checked, mask = check(people, mask)
            if min_all_killed(people):
                cnt_L += 1
                break
            if L not in people:
                cnt_M += 1
                break
        # 如果没有狼被预言家查到，投票
        if not successfully_checked:
            people, mask = vote(people, mask)
cnt_L, cnt_M

100%|████████████████████████████████████████████████████████████████| 100000/100000 [00:32<00:00, 3057.55it/s]


(59772, 40228)